<a href="https://colab.research.google.com/github/jonathas-1993/Topicos_UFAM/blob/main/atividades/4_TextVectorization_tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %%
# ============================================================
# 🔍 Análise de Sentimentos no Dataset IMDB usando TF-IDF
# Autor: Jonathas Tavares Neves
# Contexto: Aula de Aprendizado de Máquina
# ============================================================
# Objetivo:
# Demonstrar passo a passo:
# - Download e preparação do dataset IMDB
# - Separação em treino, validação e teste
# - Vetorização de texto usando TF-IDF
# - Treinamento de um modelo de rede neural densa
# - Avaliação e registro de resultados de acurácia
# - Atividade de sala: cada aluno registra suas próprias simulações

In [2]:
# %%
# ============================================================
# 🧠 Importação das Bibliotecas
# ============================================================

# Bibliotecas para manipulação de arquivos e diretórios
import os
import pathlib
import random
import shutil

# Bibliotecas para construção e treinamento de modelos
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Biblioteca para registrar resultados em tabela
import pandas as pd


In [3]:
# %%
# ============================================================
# 📦 Download e Extração do Dataset IMDB
# ============================================================

# Baixa o dataset IMDB (resenhas de filmes rotuladas)
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

# Descompacta o arquivo
!tar -xf aclImdb_v1.tar.gz

# Remove os reviews não rotulados
!rm -r aclImdb/train/unsup


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  5804k      0  0:00:14  0:00:14 --:--:-- 12.7M


In [4]:
# %%
# ============================================================
# 🧩 Separação dos Dados de Treino e Validação (20% para validação)
# ============================================================

# Definindo diretórios principais
base_dir = pathlib.Path("aclImdb")
train_dir = base_dir / "train"
val_dir = base_dir / "val"

# Para cada categoria de sentimento (negativo/positivo)
for category in ("neg", "pos"):
    # Cria a pasta de validação se não existir
    os.makedirs(val_dir / category, exist_ok=True)

    # Lista todos os arquivos da categoria no treino
    files = os.listdir(train_dir / category)

    # Embaralha os arquivos de forma reprodutível (seed fixa)
    random.Random(1337).shuffle(files)

    # Calcula 20% do total para validação
    num_val_samples = int(0.2 * len(files))

    # Seleciona os últimos 20% como validação
    val_files = files[-num_val_samples:]

    # Move os arquivos selecionados para a pasta de validação
    for fname in val_files:
        shutil.move(train_dir / category / fname, val_dir / category / fname)


In [5]:
# %%
# ============================================================
# 📚 Criação dos Datasets TensorFlow
# ============================================================

batch_size = 32  # Quantidade de amostras processadas por vez

# Dataset de treino
train_ds = keras.utils.text_dataset_from_directory(
    train_dir, batch_size=batch_size
)

# Dataset de validação
val_ds = keras.utils.text_dataset_from_directory(
    val_dir, batch_size=batch_size
)

# Dataset de teste
test_ds = keras.utils.text_dataset_from_directory(
    base_dir / "test", batch_size=batch_size
)


Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [6]:
# %%
# ============================================================
# 🧮 Vetorização com TF-IDF
# ============================================================

max_tokens = 20000  # Limite de palavras no vocabulário

# Cria vetor TF-IDF
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="tf_idf"  # saída é vetor TF-IDF
)

# Criar dataset só com textos de treino
text_only_train_ds = train_ds.map(lambda x, y: x)

# Ajusta o vocabulário TF-IDF
text_vectorization.adapt(text_only_train_ds)

# Aplica TF-IDF nos datasets
tfidf_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
tfidf_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
tfidf_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)


In [7]:
# %%
# ============================================================
# 🧩 Definição do Modelo de Rede Neural
# ============================================================

def get_model(max_tokens=20000, hidden_dim=16):
    """
    Cria modelo neural simples:
    - Entrada: vetor TF-IDF
    - Camada oculta densa + ReLU
    - Dropout para regularização
    - Saída sigmoid para classificação binária
    """
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer="rmsprop",
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model

# Testa o modelo
model = get_model(max_tokens=max_tokens)
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 20000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │       320,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 320,033 (1.22 MB)

 Trainable params: 320,033 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# %%
# ============================================================
# 💾 Função de Treino e Registro de Resultados
# ============================================================

# Cria DataFrame para armazenar resultados das simulações
resultados = pd.DataFrame(columns=["Épocas", "Dimensão Oculta", "Dropout", "Acurácia (Teste)"])

def rodar_simulacao(epochs=5, hidden_dim=16, dropout=0.5):
    """
    Função para treinar modelo com parâmetros específicos e registrar
    automaticamente a acurácia final no DataFrame global 'resultados'.
    """
    global resultados  # Permite atualizar DataFrame externo

    # Cria modelo com número de neurônios na camada oculta
    model = get_model(max_tokens=max_tokens, hidden_dim=hidden_dim)

    # Atualiza taxa de dropout (camada Dropout)
    model.layers[2].rate = dropout

    # Callback para salvar melhores pesos
    checkpoint_filepath = '/tmp/checkpoint.weights.h5'
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True
    )

    # Treina o modelo
    history = model.fit(
        tfidf_train_ds.cache(),
        validation_data=tfidf_val_ds.cache(),
        epochs=epochs,
        callbacks=[checkpoint_cb],
        verbose=0
    )

    # Carrega os melhores pesos
    model.load_weights(checkpoint_filepath)

    # Avalia no conjunto de teste
    test_loss, test_acc = model.evaluate(tfidf_test_ds, verbose=0)

    # Adiciona resultados ao DataFrame
    resultados = pd.concat([resultados, pd.DataFrame([{
        "Épocas": epochs,
        "Dimensão Oculta": hidden_dim,
        "Dropout": dropout,
        "Acurácia (Teste)": round(test_acc, 3)
    }])], ignore_index=True)

    # Mostra resultado desta simulação
    print(f"✅ Simulação concluída: Épocas={epochs}, Hidden={hidden_dim}, Dropout={dropout} -> Acurácia Teste={test_acc:.3f}")


In [10]:
# %%
# ============================================================
# 🚀 Exemplos de Simulações Atualizadas
# ============================================================

# Variáveis globais para armazenar acurácias prévias
acuracia_unigram = 0.883  # Exemplo: resultado do bagunigram
acuracia_baggram = 0.898   # Exemplo: resultado do bagbigram

# Alterarando os parâmetros abaixo para testar diferentes configurações
rodar_simulacao(epochs=5, hidden_dim=16, dropout=0.5)
rodar_simulacao(epochs=10, hidden_dim=32, dropout=0.3)
rodar_simulacao(epochs=15, hidden_dim=64, dropout=0.5)

# Adiciona as acurácias do Unigram e Baggram para comparação
resultados = pd.concat([resultados, pd.DataFrame([
    {"Épocas": "-", "Dimensão Oculta": "-", "Dropout": "-", "Acurácia (Teste)": acuracia_unigram, "Modelo": "Unigram (1-gram)"},
    {"Épocas": "-", "Dimensão Oculta": "-", "Dropout": "-", "Acurácia (Teste)": acuracia_baggram, "Modelo": "Baggram (2-gram)"}
])], ignore_index=True)

# Para as simulações atuais de TF-IDF, adiciona coluna "Modelo" com valor "TF-IDF"
resultados.loc[resultados["Modelo"].isna(), "Modelo"] = "TF-IDF"

# Exibe tabela completa com todas as simulações e comparações
resultados


✅ Simulação concluída: Épocas=5, Hidden=16, Dropout=0.5 -> Acurácia Teste=0.888
✅ Simulação concluída: Épocas=10, Hidden=32, Dropout=0.3 -> Acurácia Teste=0.888
✅ Simulação concluída: Épocas=15, Hidden=64, Dropout=0.5 -> Acurácia Teste=0.888


,Épocas,Dimensão Oculta,Dropout,Acurácia (Teste),Modelo
0,5,16,0.5,0.884,TF-IDF
1,10,32,0.3,0.880,TF-IDF
2,15,64,0.5,0.885,TF-IDF
3,-,-,-,0.883,Unigram (1-gram)
4,-,-,-,0.892,Baggram (2-gram)
5,5,16,0.5,0.888,TF-IDF
6,10,32,0.3,0.888,TF-IDF
7,15,64,0.5,0.888,TF-IDF
8,-,-,-,0.883,Unigram (1-gram)
9,-,-,-,0.898,Baggram (2-gram)
